In [3]:
%pip install gradio requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
amadeus_api_key = 'ynjYWaJMJPpyeq6DPZUTot4YbgxQWkXW'
amadeus_api_secret = 'Ke7qDDDuuLLB8AA5'

In [3]:
import requests

# Replace these with your actual Amadeus API credentials
amadeus_api_key = amadeus_api_key
amadeus_api_secret = amadeus_api_secret

def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "client_credentials",
        "client_id": amadeus_api_key,
        "client_secret": amadeus_api_secret
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token: {response.status_code} - {response.text}")

try:
    access_token = get_access_token()
    print("Access Token:", access_token)
except Exception as e:
    print(e)


Access Token: cqGtOLBQwVYbnIJuAr3ALI2b4RvG


In [16]:
import gradio as gr
import requests
import re

amadeus_api_key = amadeus_api_key
amadeus_api_secret = amadeus_api_secret

def get_access_token():
    url = "https://test.api.amadeus.com/v1/security/oauth2/token"
    
    headers = {
        "Content-Type": "application/x-www-form-urlencoded"
    }
    
    data = {
        "grant_type": "client_credentials",
        "client_id": amadeus_api_key,
        "client_secret": amadeus_api_secret
    }
    
    response = requests.post(url, headers=headers, data=data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Failed to get access token: {response.status_code} - {response.text}")

def extract_info(query):
    today_pattern = r"today"
    date_pattern = r"(\d{2}-\d{2}-\d{4})"

    origin = "JKT"  # Default origin can be Jakarta
    destination = "NRT"  # Default destination can be Japan (Narita)

    if re.search(today_pattern, query, re.IGNORECASE):
        departure_date = "2024-10-09"  # Replace with actual date logic
    else:
        date_match = re.search(date_pattern, query)
        departure_date = date_match.group(1) if date_match else None

    return origin, destination, departure_date

def search_cheapest_flight(origin, destination, departure_date):
    access_token = get_access_token()  # Get a valid access token
    url = "https://test.api.amadeus.com/v2/shopping/flight-offers"
    
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    
    params = {
        "originLocationCode": origin,
        "destinationLocationCode": destination,
        "departureDate": departure_date,
        "adults": 1,
        "maxPrice": 10000
    }

    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        return response.json()  # Valid JSON response
    else:
        return {"error": f"Error: {response.status_code} - {response.json()}"}

def chatbot_interface(query):
    origin, destination, departure_date = extract_info(query)
    if departure_date is None:
        return {"error": "Please provide a valid departure date or ask for today's flights."}
    return search_cheapest_flight(origin, destination, departure_date)

iface = gr.Interface(fn=chatbot_interface,
                     inputs="text",  # Single text input
                     outputs="json",
                     title="Travel Assistant Chatbot",
                     description="Find the cheapest flight tickets by asking questions.")

iface.launch()


Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.
